In [14]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
df=pd.read_csv ("diabetic_data.csv")

In [15]:
df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


df.shape

In [16]:
df['readmitted'].unique()

array(['NO', '>30', '<30'], dtype=object)

In [17]:
df['readmitted']=df['readmitted'].replace(['NO','>30','<30'],['0','0','1'])
df['readmitted']=df['readmitted'].astype('int')
df['readmitted']

0         0
1         0
2         0
3         0
4         0
         ..
101761    0
101762    0
101763    0
101764    0
101765    0
Name: readmitted, Length: 101766, dtype: int64

In [18]:
df['weight'].value_counts()

weight
?            98569
[75-100)      1336
[50-75)        897
[100-125)      625
[125-150)      145
[25-50)         97
[0-25)          48
[150-175)       35
[175-200)       11
>200             3
Name: count, dtype: int64

dropping encounter_id,patient_nbr as it is just identifier 
weight as it contains 97% null values
examide, citogliption as they have only one categorical value
diag_1,diag_2,diag_3 as we are considering number_of_diagnosis

In [19]:
df=df.drop(['encounter_id','patient_nbr','weight','examide','citoglipton','diag_1','diag_2','diag_3'],axis=1)

In [20]:
df.replace('?',np.nan,inplace=True)

In [21]:
df.isnull().sum()

race                         2273
gender                          0
age                             0
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
payer_code                  40256
medical_specialty           49949
num_lab_procedures              0
num_procedures                  0
num_medications                 0
number_outpatient               0
number_emergency                0
number_inpatient                0
number_diagnoses                0
max_glu_serum               96420
A1Cresult                   84748
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide                   0
glipizide                       0
glyburide                       0
tolbutamide                     0
pioglitazone                    0
rosiglitazone                   0
acarbose      

In [22]:
#discharge_disposition_id informs where the patient has gone after discharging. Ids 11,13,14,19,20,21 represent death or Hospice they cannot be returned
df=df.loc[~df.discharge_disposition_id.isin([11,13,14,19,20,21])]

In [23]:
#Numerical Columns in dataset
df_num_cols=df[['time_in_hospital','num_lab_procedures','num_procedures','num_medications','number_outpatient','number_emergency','number_inpatient','number_diagnoses']]
df_num_cols.isnull().sum()

time_in_hospital      0
num_lab_procedures    0
num_procedures        0
num_medications       0
number_outpatient     0
number_emergency      0
number_inpatient      0
number_diagnoses      0
dtype: int64

In [24]:
df_num_cols.corr()

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
time_in_hospital,1.000000,0.319855,0.189908,0.463757,-0.009214,-0.009573,0.074251,0.220600
num_lab_procedures,0.319855,1.000000,0.051715,0.265148,-0.007610,-0.000980,0.039908,0.149663
num_procedures,0.189908,0.051715,1.000000,0.381931,-0.025263,-0.038715,-0.067021,0.070460
num_medications,0.463757,0.265148,0.381931,1.000000,0.046037,0.014028,0.065266,0.261639
number_outpatient,-0.009214,-0.007610,-0.025263,0.046037,1.000000,0.091668,0.108015,0.094829
number_emergency,-0.009573,-0.000980,-0.038715,0.014028,0.091668,1.000000,0.267702,0.055994
number_inpatient,0.074251,0.039908,-0.067021,0.065266,0.108015,0.267702,1.000000,0.104737
number_diagnoses,0.220600,0.149663,0.070460,0.261639,0.094829,0.055994,0.104737,1.000000


In [25]:
#Categorical Columns in dataset
categorical_columns=['race', 'gender','payer_code','max_glu_serum','medical_specialty', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
       'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose',
       'miglitol', 'troglitazone', 'tolazamide', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed']

df[categorical_columns].isnull().sum()

race                         2234
gender                          0
payer_code                  39398
max_glu_serum               94191
medical_specialty           48616
A1Cresult                   82509
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide                   0
glipizide                       0
glyburide                       0
tolbutamide                     0
pioglitazone                    0
rosiglitazone                   0
acarbose                        0
miglitol                        0
troglitazone                    0
tolazamide                      0
insulin                         0
glyburide-metformin             0
glipizide-metformin             0
glimepiride-pioglitazone        0
metformin-rosiglitazone         0
metformin-pioglitazone          0
change                          0
diabetesMed                     0
dtype: int64

In [26]:
df['race']=df['race'].fillna('UKN')
df['medical_specialty']=df['medical_specialty'].fillna('UKN')
df['payer_code']=df['payer_code'].fillna('UKN')


In [27]:
df.head()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,...,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,Caucasian,Female,[0-10),6,25,1,1,UKN,Pediatrics-Endocrinology,41,...,No,No,No,No,No,No,No,No,No,0
1,Caucasian,Female,[10-20),1,1,7,3,UKN,UKN,59,...,No,Up,No,No,No,No,No,Ch,Yes,0
2,AfricanAmerican,Female,[20-30),1,1,7,2,UKN,UKN,11,...,No,No,No,No,No,No,No,No,Yes,0
3,Caucasian,Male,[30-40),1,1,7,2,UKN,UKN,44,...,No,Up,No,No,No,No,No,Ch,Yes,0
4,Caucasian,Male,[40-50),1,1,7,1,UKN,UKN,51,...,No,Steady,No,No,No,No,No,Ch,Yes,0


In [28]:
grouped_df=df.groupby('medical_specialty').size().sort_values(ascending=False)

In [29]:
grouped_df.head(10)

medical_specialty
UKN                           48616
InternalMedicine              14237
Emergency/Trauma               7419
Family/GeneralPractice         7252
Cardiology                     5279
Surgery-General                3059
Nephrology                     1539
Orthopedics                    1392
Orthopedics-Reconstructive     1230
Radiologist                    1121
dtype: int64

In [30]:
top_10=grouped_df.head(10).index
top_10

Index(['UKN', 'InternalMedicine', 'Emergency/Trauma', 'Family/GeneralPractice',
       'Cardiology', 'Surgery-General', 'Nephrology', 'Orthopedics',
       'Orthopedics-Reconstructive', 'Radiologist'],
      dtype='object', name='medical_specialty')

In [31]:
df.loc[~df.medical_specialty.isin(top_10),'medical_specialty']= 'Other'

In [32]:
df.describe()

,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,readmitted
count,99343.000000,99343.000000,99343.000000,99343.000000,99343.000000,99343.000000,99343.000000,99343.000000,99343.000000,99343.000000,99343.000000,99343.000000
mean,2.030259,3.517882,5.731083,4.379332,42.906929,1.334236,15.979062,0.369246,0.198444,0.630935,7.401709,0.113888
std,1.446373,5.184359,4.065029,2.968409,19.610032,1.702786,8.094909,1.265142,0.937734,1.260428,1.941013,0.317677
min,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,1.000000,1.000000,1.000000,2.000000,31.000000,0.000000,10.000000,0.000000,0.000000,0.000000,6.000000,0.000000
50%,1.000000,1.000000,7.000000,4.000000,44.000000,1.000000,15.000000,0.000000,0.000000,0.000000,8.000000,0.000000
75%,3.000000,3.000000,7.000000,6.000000,57.000000,2.000000,20.000000,0.000000,0.000000,1.000000,9.000000,0.000000
max,8.000000,28.000000,25.000000,14.000000,132.000000,6.000000,81.000000,42.000000,76.000000,21.000000,16.000000,1.000000


In [33]:
df['admission_type_id']=df['admission_type_id'].astype('str')
df['discharge_disposition_id']=df['discharge_disposition_id'].astype('str')
df['admission_source_id']=df['admission_source_id'].astype('str')

In [34]:
df['admission_type_id'].unique()

array(['6', '1', '2', '3', '4', '5', '8', '7'], dtype=object)

In [35]:
other_columns=['admission_type_id','discharge_disposition_id','admission_source_id','medical_specialty']
df_cat=pd.get_dummies(df[categorical_columns+ other_columns],drop_first=True,dtype=int)

In [36]:
list(df_cat)

['race_Asian',
 'race_Caucasian',
 'race_Hispanic',
 'race_Other',
 'race_UKN',
 'gender_Male',
 'gender_Unknown/Invalid',
 'payer_code_CH',
 'payer_code_CM',
 'payer_code_CP',
 'payer_code_DM',
 'payer_code_FR',
 'payer_code_HM',
 'payer_code_MC',
 'payer_code_MD',
 'payer_code_MP',
 'payer_code_OG',
 'payer_code_OT',
 'payer_code_PO',
 'payer_code_SI',
 'payer_code_SP',
 'payer_code_UKN',
 'payer_code_UN',
 'payer_code_WC',
 'max_glu_serum_>300',
 'max_glu_serum_Norm',
 'medical_specialty_Emergency/Trauma',
 'medical_specialty_Family/GeneralPractice',
 'medical_specialty_InternalMedicine',
 'medical_specialty_Nephrology',
 'medical_specialty_Orthopedics',
 'medical_specialty_Orthopedics-Reconstructive',
 'medical_specialty_Other',
 'medical_specialty_Radiologist',
 'medical_specialty_Surgery-General',
 'medical_specialty_UKN',
 'A1Cresult_>8',
 'A1Cresult_Norm',
 'metformin_No',
 'metformin_Steady',
 'metformin_Up',
 'repaglinide_No',
 'repaglinide_Steady',
 'repaglinide_Up',
 'nateg

In [37]:
df_cat

,race_Asian,race_Caucasian,race_Hispanic,race_Other,race_UKN,gender_Male,gender_Unknown/Invalid,payer_code_CH,payer_code_CM,payer_code_CP,...,medical_specialty_Emergency/Trauma,medical_specialty_Family/GeneralPractice,medical_specialty_InternalMedicine,medical_specialty_Nephrology,medical_specialty_Orthopedics,medical_specialty_Orthopedics-Reconstructive,medical_specialty_Other,medical_specialty_Radiologist,medical_specialty_Surgery-General,medical_specialty_UKN
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
101762,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
101763,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
101764,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [38]:
df_num_cols

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
0,1,41,0,1,0,0,0,1
1,3,59,0,18,0,0,0,9
2,2,11,5,13,2,0,1,6
3,2,44,1,16,0,0,0,7
4,1,51,0,8,0,0,0,5
...,...,...,...,...,...,...,...,...
101761,3,51,0,16,0,0,0,9
101762,5,33,3,18,0,0,1,9
101763,1,53,0,9,1,0,0,13
101764,10,45,2,21,0,0,1,9


In [39]:
df['age']

0          [0-10)
1         [10-20)
2         [20-30)
3         [30-40)
4         [40-50)
           ...   
101761    [70-80)
101762    [80-90)
101763    [70-80)
101764    [80-90)
101765    [70-80)
Name: age, Length: 99343, dtype: object

In [40]:
df_2=pd.concat([df_cat,df_num_cols,df['age']],axis=1)

In [41]:
len(df_2)

99343

In [42]:
df_2.head()

,race_Asian,race_Caucasian,race_Hispanic,race_Other,race_UKN,gender_Male,gender_Unknown/Invalid,payer_code_CH,payer_code_CM,payer_code_CP,...,medical_specialty_UKN,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,age
0,0,1,0,0,0,0,0,0,0,0,...,0,1,41,0,1,0,0,0,1,[0-10)
1,0,1,0,0,0,0,0,0,0,0,...,1,3,59,0,18,0,0,0,9,[10-20)
2,0,0,0,0,0,0,0,0,0,0,...,1,2,11,5,13,2,0,1,6,[20-30)
3,0,1,0,0,0,1,0,0,0,0,...,1,2,44,1,16,0,0,0,7,[30-40)
4,0,1,0,0,0,1,0,0,0,0,...,1,1,51,0,8,0,0,0,5,[40-50)


In [43]:
df_2.dtypes

race_Asian            int64
race_Caucasian        int64
race_Hispanic         int64
race_Other            int64
race_UKN              int64
                      ...  
number_outpatient     int64
number_emergency      int64
number_inpatient      int64
number_diagnoses      int64
age                  object
Length: 150, dtype: object

In [44]:
age_id = {'[0-10)':5, 
          '[10-20)':15, 
          '[20-30)':25, 
          '[30-40)':35, 
          '[40-50)':45, 
          '[50-60)':55,
          '[60-70)':65, 
          '[70-80)':75, 
          '[80-90)':85, 
          '[90-100)':95}
df_2['age_group'] = df_2.age.replace(age_id)

In [45]:
df_2.columns

Index(['race_Asian', 'race_Caucasian', 'race_Hispanic', 'race_Other',
       'race_UKN', 'gender_Male', 'gender_Unknown/Invalid', 'payer_code_CH',
       'payer_code_CM', 'payer_code_CP',
       ...
       'time_in_hospital', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'number_diagnoses', 'age', 'age_group'],
      dtype='object', length=151)

In [46]:
df_2=pd.concat([df_2,df['readmitted']],axis=1)

In [47]:
df_2.head()

,race_Asian,race_Caucasian,race_Hispanic,race_Other,race_UKN,gender_Male,gender_Unknown/Invalid,payer_code_CH,payer_code_CM,payer_code_CP,...,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,age,age_group,readmitted
0,0,1,0,0,0,0,0,0,0,0,...,41,0,1,0,0,0,1,[0-10),5,0
1,0,1,0,0,0,0,0,0,0,0,...,59,0,18,0,0,0,9,[10-20),15,0
2,0,0,0,0,0,0,0,0,0,0,...,11,5,13,2,0,1,6,[20-30),25,0
3,0,1,0,0,0,1,0,0,0,0,...,44,1,16,0,0,0,7,[30-40),35,0
4,0,1,0,0,0,1,0,0,0,0,...,51,0,8,0,0,0,5,[40-50),45,0


In [48]:
df_2.columns

Index(['race_Asian', 'race_Caucasian', 'race_Hispanic', 'race_Other',
       'race_UKN', 'gender_Male', 'gender_Unknown/Invalid', 'payer_code_CH',
       'payer_code_CM', 'payer_code_CP',
       ...
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient',
       'number_diagnoses', 'age', 'age_group', 'readmitted'],
      dtype='object', length=152)

In [49]:
df_2.dtypes

race_Asian           int64
race_Caucasian       int64
race_Hispanic        int64
race_Other           int64
race_UKN             int64
                     ...  
number_inpatient     int64
number_diagnoses     int64
age                 object
age_group            int64
readmitted           int64
Length: 152, dtype: object

In [50]:
df_2=df_2.drop(['age'],axis=1)

In [51]:
df_2.dtypes

race_Asian          int64
race_Caucasian      int64
race_Hispanic       int64
race_Other          int64
race_UKN            int64
                    ...  
number_emergency    int64
number_inpatient    int64
number_diagnoses    int64
age_group           int64
readmitted          int64
Length: 151, dtype: object

In [52]:
df_final=df_2.copy()
df_final.head()

,race_Asian,race_Caucasian,race_Hispanic,race_Other,race_UKN,gender_Male,gender_Unknown/Invalid,payer_code_CH,payer_code_CM,payer_code_CP,...,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,age_group,readmitted
0,0,1,0,0,0,0,0,0,0,0,...,1,41,0,1,0,0,0,1,5,0
1,0,1,0,0,0,0,0,0,0,0,...,3,59,0,18,0,0,0,9,15,0
2,0,0,0,0,0,0,0,0,0,0,...,2,11,5,13,2,0,1,6,25,0
3,0,1,0,0,0,1,0,0,0,0,...,2,44,1,16,0,0,0,7,35,0
4,0,1,0,0,0,1,0,0,0,0,...,1,51,0,8,0,0,0,5,45,0


In [53]:
df_final.corr()

,race_Asian,race_Caucasian,race_Hispanic,race_Other,race_UKN,gender_Male,gender_Unknown/Invalid,payer_code_CH,payer_code_CM,payer_code_CP,...,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,age_group,readmitted
race_Asian,1.000000,-0.137092,-0.011482,-0.009782,-0.012098,0.005587,-0.000438,0.000344,-0.001757,-0.008644,...,-0.011305,-0.007866,0.000007,-0.027157,-0.013140,-0.009159,-0.010603,-0.015264,0.001773,-0.002607
race_Caucasian,-0.137092,1.000000,-0.247436,-0.210791,-0.260697,0.055796,-0.009445,0.002397,-0.017092,0.019741,...,-0.006125,-0.024138,0.032885,0.057437,0.058531,-0.021769,-0.002150,0.109863,0.168633,0.007524
race_Hispanic,-0.011482,-0.247436,1.000000,-0.017655,-0.021835,-0.000092,-0.000791,-0.003557,-0.004804,0.001804,...,-0.015944,-0.001632,-0.019246,-0.035363,-0.004726,0.004851,-0.002016,-0.036339,-0.063343,-0.003980
race_Other,-0.009782,-0.210791,-0.017655,1.000000,-0.018601,0.009541,0.014487,-0.002435,-0.002358,-0.006179,...,-0.005035,0.001678,0.014092,-0.013289,-0.010374,0.005943,-0.017235,-0.015509,-0.025632,-0.006201
race_UKN,-0.012098,-0.260697,-0.021835,-0.018601,1.000000,0.011184,0.023877,-0.002144,-0.013591,0.005038,...,-0.004356,0.008451,-0.004054,-0.003902,-0.021031,-0.024857,-0.039403,-0.057624,0.005907,-0.014197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
number_emergency,-0.009159,-0.021769,0.004851,0.005943,-0.024857,-0.021463,-0.001163,-0.002608,0.010551,-0.013962,...,-0.009573,-0.000980,-0.038715,0.014028,0.091668,1.000000,0.267702,0.055994,-0.087568,0.060885
number_inpatient,-0.010603,-0.002150,-0.002016,-0.017235,-0.039403,-0.015188,-0.002751,-0.010061,-0.006691,-0.025608,...,0.074251,0.039908,-0.067021,0.065266,0.108015,0.267702,1.000000,0.104737,-0.044337,0.168326
number_diagnoses,-0.015264,0.109863,-0.036339,-0.015509,-0.057624,-0.003421,-0.003969,0.006858,0.032343,0.008346,...,0.220600,0.149663,0.070460,0.261639,0.094829,0.055994,0.104737,1.000000,0.241269,0.053580
age_group,0.001773,0.168633,-0.063343,-0.025632,0.005907,-0.050515,0.002041,-0.008617,0.045918,-0.081214,...,0.107077,0.016323,-0.027885,0.043166,0.023671,-0.087568,-0.044337,0.241269,1.000000,0.022196


In [54]:
df_sample = df_final.sample(n = len(df_final))
df_sample = df_sample.reset_index(drop = True)
df_sample

,race_Asian,race_Caucasian,race_Hispanic,race_Other,race_UKN,gender_Male,gender_Unknown/Invalid,payer_code_CH,payer_code_CM,payer_code_CP,...,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,age_group,readmitted
0,0,1,0,0,0,0,0,0,0,0,...,3,32,0,13,0,0,0,3,55,0
1,0,0,0,0,0,0,0,0,0,0,...,3,46,0,7,0,0,0,5,85,0
2,0,0,0,0,0,0,0,0,0,0,...,8,33,3,10,0,0,0,9,85,0
3,1,0,0,0,0,1,0,0,0,0,...,2,39,1,18,0,0,0,7,65,0
4,0,0,0,0,0,0,0,0,0,0,...,14,77,3,30,2,0,3,9,75,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99338,0,0,0,0,0,0,0,0,0,0,...,5,43,3,33,0,0,0,5,55,0
99339,0,1,0,0,0,0,0,0,0,0,...,12,30,0,17,0,0,0,7,85,0
99340,0,1,0,0,0,1,0,0,0,0,...,5,56,1,12,0,0,0,9,55,0
99341,0,1,0,0,0,0,0,0,0,0,...,1,59,3,8,0,0,1,7,45,1


In [55]:
from sklearn.model_selection import train_test_split

# Step 1: Split into train (70%) and temp (30%)
df_train, df_temp = train_test_split(df_sample, test_size=0.3, random_state=42)

# Step 2: Split temp (30%) into valid (15%) and test (15%)
df_valid,df_test = train_test_split(df_temp, test_size=0.5, random_state=42)

In [56]:
def calc_prevalence(y_actual):
    return (sum(y_actual)/len(y_actual))

In [57]:
print(f"Test prevalence (n={len(df_test)}): {calc_prevalence(df_test['readmitted']):.3f}")
print(f"Valid prevalence (n={len(df_valid)}): {calc_prevalence(df_valid['readmitted']):.3f}")
print(f"Train prevalence (n={len(df_train)}): {calc_prevalence(df_train['readmitted']):.3f}")

Test prevalence (n=14902): 0.111
Valid prevalence (n=14901): 0.115
Train prevalence (n=69540): 0.114


In [58]:
rows_positive = df_train.readmitted == 1
df_train_pos = df_train[df_train["readmitted"] == 1]
df_train_neg = df_train[df_train["readmitted"] != 1]

# merge the balanced data
df_train = pd.concat([df_train_pos, df_train_neg.sample(n = len(df_train_pos))],axis = 0)

# shuffle the order of training samples 
df_train = df_train.sample(n = len(df_train)).reset_index(drop = True)


In [59]:
df_train

,race_Asian,race_Caucasian,race_Hispanic,race_Other,race_UKN,gender_Male,gender_Unknown/Invalid,payer_code_CH,payer_code_CM,payer_code_CP,...,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,age_group,readmitted
0,0,1,0,0,0,1,0,0,0,0,...,5,57,3,16,0,0,0,5,55,1
1,0,1,0,0,0,0,0,0,0,0,...,2,45,1,11,0,0,1,5,75,1
2,0,1,0,0,0,1,0,0,0,0,...,1,9,0,5,0,1,0,9,85,1
3,0,1,0,0,0,0,0,0,0,0,...,2,21,3,9,0,0,0,3,65,0
4,0,1,0,0,0,0,0,0,0,0,...,7,36,2,28,0,0,0,4,55,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15877,0,1,0,0,0,0,0,0,0,0,...,2,36,1,9,2,1,1,4,75,1
15878,0,1,0,0,0,1,0,0,0,0,...,3,30,1,7,1,0,1,7,55,1
15879,0,1,0,0,0,0,0,0,0,0,...,5,73,0,29,2,0,2,9,45,1
15880,0,1,0,0,0,1,0,0,0,0,...,6,96,2,13,1,0,0,9,65,0


In [60]:
X_train = df_train.drop(columns=["readmitted"],axis=1)
y_train=df_train['readmitted']
X_valid =df_valid.drop(columns=["readmitted"],axis=1)
y_valid=df_valid['readmitted']

In [61]:
X_test = df_test.drop(columns=["readmitted"],axis=1)
y_test=df_test['readmitted']

In [62]:
from sklearn.preprocessing import StandardScaler

scaler  = StandardScaler()
scaler.fit(X_train)

,copy,True
,with_mean,True
,with_std,True


In [63]:
df_train.describe()

,race_Asian,race_Caucasian,race_Hispanic,race_Other,race_UKN,gender_Male,gender_Unknown/Invalid,payer_code_CH,payer_code_CM,payer_code_CP,...,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,age_group,readmitted
count,15882.000000,15882.000000,15882.000000,15882.000000,15882.000000,15882.000000,15882.0,15882.000000,15882.000000,15882.000000,...,15882.000000,15882.000000,15882.000000,15882.000000,15882.000000,15882.000000,15882.000000,15882.000000,15882.000000,15882.000000
mean,0.006171,0.750913,0.020149,0.013600,0.018889,0.465055,0.0,0.001259,0.018889,0.023045,...,4.546783,43.516245,1.310666,16.327352,0.397557,0.262121,0.903602,7.537401,66.116358,0.500000
std,0.078312,0.432498,0.140513,0.115828,0.136138,0.498793,0.0,0.035465,0.136138,0.150051,...,3.013202,19.467766,1.670917,8.097670,1.259579,1.002101,1.645249,1.869562,15.930132,0.500016
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,5.000000,0.000000
25%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,2.000000,32.000000,0.000000,11.000000,0.000000,0.000000,0.000000,6.000000,55.000000,0.000000
50%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,4.000000,44.000000,1.000000,15.000000,0.000000,0.000000,0.000000,9.000000,65.000000,0.500000
75%,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.000000,0.000000,...,6.000000,57.000000,2.000000,20.000000,0.000000,0.000000,1.000000,9.000000,75.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,...,14.000000,132.000000,6.000000,72.000000,40.000000,25.000000,21.000000,16.000000,95.000000,1.000000


In [64]:
X_train_tf = scaler.transform(X_train)
X_valid_tf = scaler.transform(X_valid)

In [65]:
X_test_tf = scaler.transform(X_test)

In [66]:
y_test

90217    0
56368    0
78373    0
94256    0
8160     1
        ..
66784    0
29306    0
18147    0
41771    0
80349    1
Name: readmitted, Length: 14902, dtype: int64

In [67]:
pd.DataFrame(X_train_tf)[9].unique()

array([-0.15358566,  6.5110247 ])

In [68]:
X_train.shape

(15882, 150)

In [69]:
X_train_tf.shape

(15882, 150)

In [70]:
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
def calc_specificity(y_actual, y_pred, thresh):
    # calculates specificity
    return sum((y_pred < thresh) & (y_actual == 0)) /sum(y_actual ==0)

def print_report(y_actual, y_pred, thresh):
    
    auc = roc_auc_score(y_actual, y_pred)
    accuracy = accuracy_score(y_actual, (y_pred > thresh))
    recall = recall_score(y_actual, (y_pred > thresh))
    precision = precision_score(y_actual, (y_pred > thresh))
    specificity = calc_specificity(y_actual, y_pred, thresh)
    print('AUC:%.3f'%auc)
    print('accuracy:%.3f'%accuracy)
    print('recall:%.3f'%recall)
    print('precision:%.3f'%precision)
    print('specificity:%.3f'%specificity)
    print('prevalence:%.3f'%calc_prevalence(y_actual))
    print(' ')
    return auc, accuracy, recall, precision, specificity 

In [71]:
thresh = 0.5

In [72]:
#LOGISTIC REGRESSION
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression(max_iter=1000)
lr.fit(X_train_tf, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [73]:
print(lr.coef_, lr.intercept_)

[[-1.51728262e-02 -1.80460994e-02 -2.61562118e-02 -9.67055766e-04
  -3.98914197e-02 -6.30773202e-04  0.00000000e+00  1.16681876e-02
  -3.98034188e-03 -2.95070799e-02 -1.24655821e-02  0.00000000e+00
  -1.93714200e-02  2.15199830e-02 -6.12276348e-03  9.48580190e-03
   2.88105441e-02 -1.17386726e-02 -2.05801754e-02  1.62239699e-02
   1.29845650e-02  8.26642083e-02 -1.86463657e-02 -4.52479325e-02
   6.24951525e-03  2.99967469e-03 -1.77243288e-02  1.52753525e-02
   1.80493319e-02  1.83515820e-02  1.34616667e-02 -1.39631512e-02
   2.05311656e-03  1.52528108e-03 -6.21466708e-03  2.91422357e-02
  -3.73006737e-02 -1.14999821e-02 -1.86634509e-01 -2.42626313e-01
  -5.30184508e-02  1.67358338e-01  1.72414450e-01  5.90738200e-02
  -1.94570439e-02 -1.26261556e-02 -5.78210589e-03  1.33017381e-02
  -1.33017381e-02  0.00000000e+00 -1.76713629e-01 -1.80646840e-01
  -4.54344343e-02  0.00000000e+00 -1.38314213e-01 -1.08549810e-01
  -2.58007446e-02  1.01368071e-01  1.03649101e-01  8.57827729e-03
  -1.90554

In [74]:
y_train_preds = lr.predict_proba(X_train_tf)[:,1]
y_valid_preds = lr.predict_proba(X_valid_tf)[:,1]

print('Logistic Regression')
print('Training:')
lr_train_auc, lr_train_accuracy, lr_train_recall, \
    lr_train_precision, lr_train_specificity = print_report(y_train,y_train_preds, thresh)
print('Validation:')
lr_valid_auc, lr_valid_accuracy, lr_valid_recall, \
    lr_valid_precision, lr_valid_specificity = print_report(y_valid,y_valid_preds, thresh)

Logistic Regression
Training:
AUC:0.677
accuracy:0.625
recall:0.554
precision:0.646
specificity:0.696
prevalence:0.500
 
Validation:
AUC:0.652
accuracy:0.663
recall:0.525
precision:0.176
specificity:0.681
prevalence:0.115
 


In [75]:
#STOCHASTIC GRADIENT DESCENT
from sklearn.linear_model import SGDClassifier
sgdc=SGDClassifier(loss = 'log_loss',alpha = 0.1,random_state = 42)
sgdc.fit(X_train_tf, y_train)

,loss,'log_loss'
,penalty,'l2'
,alpha,0.1
,l1_ratio,0.15
,fit_intercept,True
,max_iter,1000
,tol,0.001
,shuffle,True
,verbose,0
,epsilon,0.1
,n_jobs,None


In [76]:
y_train_preds = sgdc.predict_proba(X_train_tf)[:,1]
y_valid_preds = sgdc.predict_proba(X_valid_tf)[:,1]

print('Stochastic Gradient Descend')
print('Training:')
sgdc_train_auc, sgdc_train_accuracy, sgdc_train_recall, sgdc_train_precision, sgdc_train_specificity =print_report(y_train,y_train_preds, thresh)
print('Validation:')
sgdc_valid_auc, sgdc_valid_accuracy, sgdc_valid_recall, sgdc_valid_precision, sgdc_valid_specificity = print_report(y_valid,y_valid_preds, thresh)

Stochastic Gradient Descend
Training:
AUC:0.673
accuracy:0.623
recall:0.564
precision:0.639
specificity:0.682
prevalence:0.500
 
Validation:
AUC:0.650
accuracy:0.655
recall:0.538
precision:0.175
specificity:0.670
prevalence:0.115
 


In [77]:
#RandomForest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(class_weight="balanced")
rf.fit(X_train, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [78]:
y_train_preds = rf.predict_proba(X_train_tf)[:,1]
y_valid_preds = rf.predict_proba(X_valid_tf)[:,1]

print('Random Forest:')
print('Training:')
rf_train_auc, rf_train_accuracy, rf_train_recall, rf_train_precision, rf_train_specificity =print_report(y_train,y_train_preds, thresh)
print('Validation:')
rf_valid_auc, rf_valid_accuracy, rf_valid_recall, rf_valid_precision, rf_valid_specificity = print_report(y_valid,y_valid_preds, thresh)

Random Forest:
Training:
AUC:0.640
accuracy:0.549
recall:0.137
precision:0.778
specificity:0.952
prevalence:0.500
 
Validation:
AUC:0.614
accuracy:0.858
recall:0.131
precision:0.266
specificity:0.941
prevalence:0.115
 


In [79]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors = 100)
knn.fit(X_train_tf, y_train)

,n_neighbors,100
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [80]:
y_train_preds = knn.predict_proba(X_train_tf)[:,1]
y_valid_preds = knn.predict_proba(X_valid_tf)[:,1]

print('KNN')
print('Training:')
knn_train_auc, knn_train_accuracy, knn_train_recall, \
    knn_train_precision, knn_train_specificity = print_report(y_train,y_train_preds, thresh)
print('Validation:')
knn_valid_auc, knn_valid_accuracy, knn_valid_recall, \
    knn_valid_precision, knn_valid_specificity = print_report(y_valid,y_valid_preds, thresh)

KNN
Training:
AUC:0.637
accuracy:0.594
recall:0.483
precision:0.620
specificity:0.660
prevalence:0.500
 
Validation:
AUC:0.591
accuracy:0.653
recall:0.453
precision:0.155
specificity:0.633
prevalence:0.115
 


In [81]:
#NAIVE BAYES
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()
nb.fit(X_train_tf, y_train)

,priors,None
,var_smoothing,1e-09


In [82]:
y_train_preds = nb.predict_proba(X_train_tf)[:,1]
y_valid_preds = nb.predict_proba(X_valid_tf)[:,1]

print('Naive Bayes')
print('Training:')
nb_train_auc, nb_train_accuracy, nb_train_recall, nb_train_precision, nb_train_specificity =print_report(y_train,y_train_preds, thresh)
print('Validation:')
nb_valid_auc, nb_valid_accuracy, nb_valid_recall, nb_valid_precision, nb_valid_specificity = print_report(y_valid,y_valid_preds, thresh)

Naive Bayes
Training:
AUC:0.531
accuracy:0.507
recall:0.985
precision:0.504
specificity:0.030
prevalence:0.500
 
Validation:
AUC:0.513
accuracy:0.135
recall:0.979
precision:0.115
specificity:0.025
prevalence:0.115
 


In [91]:
from xgboost import XGBClassifier

# Define model
xgb_clf = XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=(y_train.value_counts()[0] / y_train.value_counts()[1]), # handle imbalance
    eval_metric="auc",
    random_state=42,
    use_label_encoder=False
)

# Fit on training data
xgb_clf.fit(X_train_tf, y_train.values)

# Predictions (probabilities)
y_train_preds = xgb_clf.predict_proba(X_train_tf)[:,1]
y_valid_preds = xgb_clf.predict_proba(X_valid_tf)[:,1]

print('XGBoost')
print('Training:')
xgb_train_auc, xgb_train_accuracy, xgb_train_recall, xgb_train_precision, xgb_train_specificity = print_report(y_train, y_train_preds, thresh)

print('Validation:')
xgb_valid_auc, xgb_valid_accuracy, xgb_valid_recall, xgb_valid_precision, xgb_valid_specificity = print_report(y_valid, y_valid_preds, thresh)

XGBoost
Training:
AUC:0.822
accuracy:0.738
recall:0.709
precision:0.754
specificity:0.768
prevalence:0.500
 
Validation:
AUC:0.657
accuracy:0.629
recall:0.593
precision:0.174
specificity:0.634
prevalence:0.115
 


In [93]:
models = {
    "SGD": sgdc,
    "KNN": knn,
    "Random Forest":rf,
    "Logistic Regression": lr,
    "GaussianNB": nb
}

results = []

for name, model in models.items():
    if hasattr(model, "predict_proba"):
        y_pred = model.predict_proba(X_test)[:, 1]
    elif hasattr(model, "decision_function"):
        y_pred = model.decision_function(X_test)
    else:
        y_pred = model.predict(X_test)
    
    auc, acc, rec, prec, spec = print_report(y_test.values, y_pred, thresh=0.5)  
    results.append({"Model": name, "AUC": auc, "Accuracy": acc, "Recall": rec, "Precision": prec, "Specificity": spec})

results_df = pd.DataFrame(results)
results_df

AUC:0.591
accuracy:0.111
recall:1.000
precision:0.111
specificity:0.000
prevalence:0.111
 
AUC:0.544
accuracy:0.142
recall:0.976
precision:0.113
specificity:0.017
prevalence:0.111
 
AUC:0.638
accuracy:0.617
recall:0.569
precision:0.159
specificity:0.597
prevalence:0.111
 
AUC:0.600
accuracy:0.112
recall:1.000
precision:0.111
specificity:0.001
prevalence:0.111
 
AUC:0.500
accuracy:0.889
recall:0.000
precision:0.000
specificity:1.000
prevalence:0.111
 


,Model,AUC,Accuracy,Recall,Precision,Specificity
0,SGD,0.590752,0.111394,1.000000,0.111394,0.000000
1,KNN,0.544348,0.141525,0.975904,0.112711,0.017369
2,Random Forest,0.638334,0.617165,0.568675,0.159110,0.597266
3,Logistic Regression,0.600251,0.112065,1.000000,0.111469,0.000755
4,GaussianNB,0.499962,0.888538,0.000000,0.000000,0.999924
